In [1]:
from clpsych import store, data
import codecs
import os
import pandas

## Read in data from parse files

In [ ]:
#currently unused

tokenized = {}
i = 0

base_i = i
more = True
while more:
    if i % 10 == 0: print i
    if i < base_i + 50 and os.path.isfile('parses/{}.parse'.format(i)):
        docs, titles = data.read_parses('parses/{}.parse'.format(i))
    
        keys = [key for key in docs.keys() if key]
        for key in keys:
            
            doc, title = [], []
            for token in docs[key]:
                doc.append(token)
            for token in titles[key]:
                title.append(token)
            tokenized[key] = (title, doc)
        i += 1
    else:
        break

0
10
20
30
40
50


## Load in data.h5

In [ ]:
#including parse takes 6 hours and doesn't do anything
config = {
    'mask': './**/**/*.posts',
    'train_mask': './**/**/TRAIN.txt',
    'test_mask': './**/**/TEST.txt',
    'dev_mask': './**/**/DEV.txt',
    'sample_mask': './SAMPLE.txt',
    #'parse':'./parses/*.parse',
    #'parsed':True
}

load = store.Store('data.h5', config=config, overwrite = False)
loaded_data = load.data

## Calculate Lengths

In [ ]:
#select subset of data
#indices = data.read_indices('SAMPLE.txt')
#load = store.Store('data.h5')
#loaded_data = load.select(indices)


In [ ]:

df = pandas.DataFrame(columns=('post_id', 'num_sent', 'avg_len',  'accuracy', 'freq_accuracy', 'len_title', 'accuracy_title', 'freq_title'))
#'spelling', 'freq', 'spelling_title', 

count = 0
for key, val in tokenized.iteritems():
    if count % 25000 == 0:
        print count
    
    #read with spacy, decode into unicode
    spacy_doc = val[1]
    spacy_title = val[0]
    
    #add tokens if their spacy rank isn't 0 (we could set a threshold here as well)
    no_typos = [tok[2] for tok in spacy_doc if tok[6]!=0]    
    infreq = [tok[2] for tok in spacy_doc if (tok[6]!=0 and tok[6]<100000)]    
    
    
    #most appear to have values over 90% (aka little typos).  Chose to set 0 length docs to 1.  
    accuracy = float(len(no_typos))/len(spacy_doc) if len(spacy_doc) > 0 else 1
    accuracy_infreq = float(len(infreq))/len(spacy_doc) if len(spacy_doc) > 0 else 1
    
    #do the same only for titles
    no_typos_title = [tok[2] for tok in spacy_title if tok[6]!=0]   
    infreq_title = [tok[2] for tok in spacy_title if (tok[6]!=0 and tok[6]<100000)]
    
    len_title =  len(spacy_title)
    #most appear to have values over 90% (aka little typos).  Chose to set 0 length docs to 1.  
    accuracy_title = float(len(no_typos_title))/len(spacy_title) if len(spacy_title) > 0 else 1
    accuracy_infreq_title = float(len(infreq_title))/len(spacy_title) if len(spacy_title) > 0 else 1
    
    #simple counts (number of sentences, and avg lengh of sentence)
    num_sent = [col[5] for col in spacy_doc].count(0)
    avg_len_sent =  len(spacy_doc)/num_sent if num_sent>0 else 0
    
    #append to new DF in the same row order
    try:
        df.loc[count] = key, num_sent, avg_len_sent, accuracy, accuracy_infreq, len_title, accuracy_title, accuracy_infreq_title
        #no_typos, infreq,no_typos_title,
    except:
        print "Issue with row ",  key
        df.loc[count] = key, 0, 0, 0, 0, 0, 0, 0
    count+=1 
print df

0
25000
50000
75000


In [ ]:
df[['user_id','post_id', 'avg_len', 'accuracy', 'freq_accuracy','len_title','num_sent','freq_title','accuracy_title']].to_csv('full_csv.csv')

In [ ]:
load = store.Store('data_features.h5', config=config, overwrite = False)
data = load.data

In [ ]:
#doc
load.create_document_feature("avg_len", df[['post_id', 'avg_len']])
load.create_document_feature("num_sent", df[['post_id', 'num_sent']])
load.create_document_feature("typo_acc", df[['post_id', 'accuracy']])
load.create_document_feature("freq_acc", df[['post_id', 'freq_accuracy']])

#title
load.create_document_feature("len_title", df[['post_id', 'len_title']])
load.create_document_feature("typo_acc_title", df[['post_id', 'accuracy_title']])
load.create_document_feature("freq_acc_title", df[['post_id', 'freq_title']])


load.finalize
load.documents

#load.documents.to_csv("full_csv.csv")

## Potpourri Analysis

In [ ]:
#can we trust .rank?  I think yes.  High ranking words are pronouns, punctuation.  Low ranking words are proper nouns.  Misspellings are generally 0s (somethines high numbers)
words = nlp(u'suicide death die I ugh hate myself me wtf asfab beelzebub shakespeare javert misppeling reddit')

[tok.rank for tok in words]


## Legacy code to run without parses



import spacy
nlp = spacy.load('en')

#this can be replaced with read_parses() if it saves "tok.sents"
#currently takes ~3 hours to run


df = pandas.DataFrame(columns=('user_id', 'post_id', 'num_sent', 'avg_len',  'accuracy', 'freq_accuracy', 'len_title', 'accuracy_title', 'freq_title'))
#'spelling', 'freq', 'spelling_title', 
count = 0 
for index, row in loaded_data.iterrows():
    index = count #index, row in itterrows resets every 10k or so!
    if (count%25000 == 0): print "Completed ", count
    #read with spacy, decode into unicode
    spacy_doc = nlp(row['text'].decode("utf-8"))
    spacy_title = nlp(row['title'].decode("utf-8"))
    
    #add tokens if their spacy rank isn't 0 (we could set a threshold here as well)
    no_typos = [tok.text for tok in spacy_doc if tok.rank!=0]    
    infreq = [tok.text for tok in spacy_doc if (tok.rank!=0 and tok.rank<100000)]    
    
    #simple counts (number of sentences, and avg lengh of sentence)
    num_sent = len(list(spacy_doc.sents))
    avg_len_sent =  sum([len(sent) for sent in list(spacy_doc.sents)])/len(list(spacy_doc.sents))  if len(list(spacy_doc.sents))> 0 else 0
    
    #most appear to have values over 90% (aka little typos).  Chose to set 0 length docs to 1.  
    accuracy = float(len(no_typos))/len(spacy_doc) if len(spacy_doc) > 0 else 1
    accuracy_infreq = float(len(infreq))/len(spacy_doc) if len(spacy_doc) > 0 else 1
    
    #do the same only for titles
    no_typos_title = [tok.text for tok in spacy_title if tok.rank!=0]   
    infreq_title = [tok.text for tok in spacy_title if (tok.rank!=0 and tok.rank<100000)]
    
    len_title =  len(spacy_title)
    #most appear to have values over 90% (aka little typos).  Chose to set 0 length docs to 1.  
    accuracy_title = float(len(no_typos_title))/len(spacy_title) if len(spacy_title) > 0 else 1
    accuracy_infreq_title = float(len(infreq_title))/len(spacy_title) if len(spacy_title) > 0 else 1
    
    #append to new DF in the same row order
    try:
        df.loc[index] = row['user_id'], row['post_id'], num_sent, avg_len_sent, accuracy, accuracy_infreq, len_title, accuracy_title, accuracy_infreq_title
        #no_typos, infreq,no_typos_title,
    except:
        print "Issue with row ",  index
        df.loc[index] = row['user_id'], row['post_id'], 0, 0, 0, 0, 0, 0
    count+=1
print df
    

#code for converting spelling into a long string.  This could be written into a DF if needed
storage = []
for row in df['spelling'][0:10]:
    string = ""
    for tok in row:
        string = string+" " + tok.text
    storage.append(string)